In [47]:
# %load jupyter_default.py
import pandas as pd
import numpy as np
import os
import re
import datetime
import time
import glob
import json
from tqdm import tqdm_notebook
from colorama import Fore, Style

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

%config InlineBackend.figure_format='retina'
sns.set() # Revert to matplotlib defaults
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.labelpad'] = 20
plt.rcParams['legend.fancybox'] = True
plt.style.use('ggplot')

SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = 14, 16, 20
plt.rc('font', size=SMALL_SIZE)
plt.rc('axes', titlesize=SMALL_SIZE)
plt.rc('axes', labelsize=MEDIUM_SIZE)
plt.rc('xtick', labelsize=SMALL_SIZE)
plt.rc('ytick', labelsize=SMALL_SIZE)
plt.rc('legend', fontsize=MEDIUM_SIZE)
plt.rc('axes', titlesize=BIGGER_SIZE)

def savefig(plt, name):
    plt.savefig(f'../../figures/{name}.png', bbox_inches='tight', dpi=300)

%reload_ext autoreload
%autoreload 2
    
%reload_ext version_information
%version_information pandas, numpy

Software versions
Python 3.6.8 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 6.4.0
OS Darwin 16.7.0 x86_64 i386 64bit
pandas 0.23.0
numpy 1.14.2
Wed Apr 17 08:42:44 2019 EDT

# GA Data Mining
Alex's development notebook for miscellaneous rough work.

### Pulling the data

> The first 1 TB of query data processed per month is free

In [11]:
from google.cloud import bigquery

In [10]:
from dotenv import load_dotenv
load_dotenv('../../.env')

True

In [16]:
client = bigquery.Client()

In [13]:
query_job = client.query("""
    SELECT
      CONCAT(
        'https://stackoverflow.com/questions/',
        CAST(id as STRING)) as url,
      view_count
    FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE tags like '%google-bigquery%'
    LIMIT 10
""")

results = query_job.result()  # Waits for job to complete.

In [14]:
results

In [15]:
for row in results:
    print("{} : {} views".format(row.url, row.view_count))

https://stackoverflow.com/questions/36363131 : 59 views
https://stackoverflow.com/questions/32208958 : 803 views
https://stackoverflow.com/questions/50769877 : 220 views
https://stackoverflow.com/questions/54931771 : 33 views
https://stackoverflow.com/questions/16044407 : 419 views
https://stackoverflow.com/questions/22135615 : 395 views
https://stackoverflow.com/questions/44970976 : 293 views
https://stackoverflow.com/questions/50935535 : 57 views
https://stackoverflow.com/questions/46738644 : 156 views
https://stackoverflow.com/questions/26201223 : 454 views


Let's select some GA data

In [18]:
query_job = client.query("""
    SELECT
        *
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
    LIMIT 10
""")

results = query_job.result()  # Waits for job to complete.

In [23]:
data = [r for r in results]

Here is one row of data

In [31]:
dict(data[0].items())

{'visitorId': None,
 'visitNumber': 1,
 'visitId': 1501583974,
 'visitStartTime': 1501583974,
 'date': '20170801',
 'totals': {'visits': 1,
  'hits': 1,
  'pageviews': 1,
  'timeOnSite': None,
  'bounces': 1,
  'transactions': None,
  'transactionRevenue': None,
  'newVisits': 1,
  'screenviews': None,
  'uniqueScreenviews': None,
  'timeOnScreen': None,
  'totalTransactionRevenue': None,
  'sessionQualityDim': 1},
 'trafficSource': {'referralPath': None,
  'campaign': '(not set)',
  'source': '(direct)',
  'medium': '(none)',
  'keyword': None,
  'adContent': None,
  'adwordsClickInfo': {'campaignId': None,
   'adGroupId': None,
   'creativeId': None,
   'criteriaId': None,
   'page': None,
   'slot': None,
   'criteriaParameters': 'not available in demo dataset',
   'gclId': None,
   'customerId': None,
   'adNetworkType': None,
   'targetingCriteria': None,
   'isVideoAd': None},
  'isTrueDirect': None,
  'campaignCode': None},
 'device': {'browser': 'Chrome',
  'browserVersion': 'n

In [32]:
len(data)

10

We got 10 total rows

Let's get some info about the dataset

In [34]:
dataset_id = 'bigquery-public-data.google_analytics_sample'
dataset = client.get_dataset(dataset_id)

In [35]:
full_dataset_id = "{}.{}".format(dataset.project, dataset.dataset_id)
friendly_name = dataset.friendly_name
print(
    "Got dataset '{}' with friendly_name '{}'.".format(
        full_dataset_id, friendly_name
    )
)

# View dataset properties
print("Description: {}".format(dataset.description))
print("Labels:")
labels = dataset.labels
if labels:
    for label, value in labels.items():
        print("\t{}: {}".format(label, value))
else:
    print("\tDataset has no labels defined.")

# View tables in dataset
print("Tables:")
tables = list(client.list_tables(dataset))  # API request(s)
if tables:
    for table in tables:
        print("\t{}".format(table.table_id))
else:
    print("\tThis dataset does not contain any tables.")


Got dataset 'bigquery-public-data.google_analytics_sample' with friendly_name 'None'.
Description: None
Labels:
	Dataset has no labels defined.
Tables:
	ga_sessions_20160801
	ga_sessions_20160802
	ga_sessions_20160803
	ga_sessions_20160804
	ga_sessions_20160805
	ga_sessions_20160806
	ga_sessions_20160807
	ga_sessions_20160808
	ga_sessions_20160809
	ga_sessions_20160810
	ga_sessions_20160811
	ga_sessions_20160812
	ga_sessions_20160813
	ga_sessions_20160814
	ga_sessions_20160815
	ga_sessions_20160816
	ga_sessions_20160817
	ga_sessions_20160818
	ga_sessions_20160819
	ga_sessions_20160820
	ga_sessions_20160821
	ga_sessions_20160822
	ga_sessions_20160823
	ga_sessions_20160824
	ga_sessions_20160825
	ga_sessions_20160826
	ga_sessions_20160827
	ga_sessions_20160828
	ga_sessions_20160829
	ga_sessions_20160830
	ga_sessions_20160831
	ga_sessions_20160901
	ga_sessions_20160902
	ga_sessions_20160903
	ga_sessions_20160904
	ga_sessions_20160905
	ga_sessions_20160906
	ga_sessions_20160907
	ga_sessions

Export the data

In [40]:
# def dump_data(table):
#     bucket_name = 'bigquery-public-data-ga-sample'
#     destination_uri = "gs://{}/{}.json".format(bucket_name, table.table_id)
#     extract_job = client.extract_table(
#         table,
#         destination_uri,
#         # Location must match that of the source table.
#         location="US",
#     )  # API request
#     extract_job.result()  # Waits for job to complete.

In [54]:
def dump_data(table_id, limit=None):
    query_job = client.query("""
        SELECT *
        FROM `bigquery-public-data.google_analytics_sample.{}`
    """.format(table_id))
    if limit is not None:
        query_job += '\nLIMIT {}'.format(limit)
    results = query_job.result()  # Waits for job to complete.
    
    with open('../../data/raw/{}.jsonl'.format(table_id), 'w') as f:
        for result in results:
            f.write('{}\n'.format(
                json.dumps(dict(result.items()))
            ))

In [50]:
dataset_id = 'bigquery-public-data.google_analytics_sample'
dataset = client.get_dataset(dataset_id)
tables = list(client.list_tables(dataset))
print('got {} tables'.format(len(tables)))

for table in tables:
    print('dumping {}'.format(table.table_id))
    dump_data(table.table_id)
    break

got 366 tables
dumping ga_sessions_20160801


In [51]:
ls ../../data/raw/

ga_sessions_20160801.jsonl


In [53]:
cat ../../data/raw/ga_sessions_20160801.jsonl | wc -l

    1711


### Saving the data locally

Pull all the data (~25gigs | 3 hours)

In [55]:
dataset_id = 'bigquery-public-data.google_analytics_sample'
dataset = client.get_dataset(dataset_id)
tables = list(client.list_tables(dataset))
print('got {} tables'.format(len(tables)))

for table in tables:
    print('dumping {}'.format(table.table_id))
    dump_data(table.table_id)

got 366 tables
dumping ga_sessions_20160801
dumping ga_sessions_20160802
dumping ga_sessions_20160803
dumping ga_sessions_20160804
dumping ga_sessions_20160805
dumping ga_sessions_20160806
dumping ga_sessions_20160807
dumping ga_sessions_20160808
dumping ga_sessions_20160809
dumping ga_sessions_20160810
dumping ga_sessions_20160811
dumping ga_sessions_20160812
dumping ga_sessions_20160813
dumping ga_sessions_20160814
dumping ga_sessions_20160815
dumping ga_sessions_20160816
dumping ga_sessions_20160817
dumping ga_sessions_20160818
dumping ga_sessions_20160819
dumping ga_sessions_20160820
dumping ga_sessions_20160821
dumping ga_sessions_20160822
dumping ga_sessions_20160823
dumping ga_sessions_20160824
dumping ga_sessions_20160825
dumping ga_sessions_20160826
dumping ga_sessions_20160827
dumping ga_sessions_20160828
dumping ga_sessions_20160829
dumping ga_sessions_20160830
dumping ga_sessions_20160831
dumping ga_sessions_20160901
dumping ga_sessions_20160902
dumping ga_sessions_20160903

In [7]:
from IPython.display import HTML
HTML('<style>div.text_cell_render{font-size:130%;padding-top:50px;padding-bottom:50px}</style>')